# ДЗ #1. Exploratory Data Analysis and Preprocessing <a id="0"></a>

### Использованный датасет: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data

* [Загрузка и первичная проверка данных](#1)
* [Базовые статистики датасета, средние, медианы, и т.д.](#2)
* [Гистограммы/распределения признаков с использованием целевой переменной](#3)
* [Heatmap для матрицы корреляций. Признаки, которые сильно скоррелированы](#4)
* [Попарные scatterplot-ы для сильно скоррелированных признаков. Наблюдается ли линейная зависимость?](#5)
* [По каким признакам наиболее удобно было бы отделить злокачественные новообразования от доброкачественных (используя boxplots и группировку по целевой переменной)](#6)

In [ ]:
# Импорт основных библиотек
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Загрузка и первичная проверка данных [(Наверх)](#0) <a id="1"></a>

### Описание признаков:
- **id**
- **diagnosis** - диагноз тканей молочной железы (M = злокачественный, B = доброкачественный)
- **radius_mean** - среднее расстояние от центра до точек по периметру
- **texture_mean** - стандартное отклонение значений gray-scale
- **perimeter_mean** - средний размер ядра опухоли
- **area_mean**
- **smoothness_mean** - среднее локального изменения длин радиуса
- **compactness_mean** - среднее perimeter^2 / area - 1.0
- **concavity_mean** - среднее тяжести вогнутых частей контура
- **concavepoints_mean** - среднее количество вогнутых частей контура
- **symmetry_mean**
- **fractal_dimension_mean** - mean for "coastline approximation" - 1
- **radius_se** - standard error for the mean of distances from center to points on the perimeter
- **texture_se** - standard error for standard deviation of gray-scale values
- **perimeter_se**
- **area_se**
- **smoothness_se** - standard error for local variation in radius lengths
- **compactness_se** - standard error for perimeter^2 / area - 1.0
- **concavity_se** - standard error for severity of concave portions of the contour
- **concave points_se** - standard error for number of concave portions of the contour
- **symmetry_se**
- **fractal_dimension_se** - standard error for "coastline approximation" - 1
- **radius_worst** - "worst" or largest mean value for mean of distances from center to points on the perimeter
- **texture_worst** - "worst" or largest mean value for standard deviation of gray-scale values
- **perimeter_worst**
- **area_worst**
- **smoothness_worst** - "worst" or largest mean value for local variation in radius lengths
- **compactness_worst** - "worst" or largest mean value for perimeter^2 / area - 1.0
- **concavity_worst** - "worst" or largest mean value for severity of concave portions of the contour
- **concave_points_worst** - "worst" or largest mean value for number of concave portions of the contour
- **symmetry_worst**
- **fractal_dimension_worst** - "worst" or largest mean value for "coastline approximation" - 1

In [ ]:
url = 'http://ftp.cs.wisc.edu/math-prog/cpo-dataset/machine-learn/cancer/WDBC/WDBC.dat'
filename = 'input/data.csv'
column_names = ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 
                'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concavepoints_mean', 
                'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 
                'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concavepoints_se', 
                'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 
                'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 
                'concave_points_worst', 'symmetry_worst', 'fractal_dimension_worst']

if not os.path.exists(filename):
    df = pd.read_csv(url, names=column_names)
    df.to_csv(filename, index=False)
else:
    df = pd.read_csv(filename)

# Проверим корректно ли загружен CSV
df.head(5)

## Базовые статистики датасета, средние, медианы, и т.д. [(Наверх)](#0) <a id="2"></a>

In [ ]:
# Узнаем размер датасета
df.shape

In [ ]:
# Проверим на наличие пустых данных
df.isnull().sum()

In [ ]:
# Узнаем свойства датасета
df.info()

In [ ]:
# Удалим ненужное поле ID
df.drop(['id'], axis=1, inplace=True)

# Сконвертируем категорийное значение 'diagnosis' в бинарный вид
diagnosis_bool = {'B':0, 'M':1}
df['diagnosis'] = df['diagnosis'].map(diagnosis_bool)

# Проверим полученный результат
df.head(5)

In [ ]:
df.describe()

## Гистограммы признаков с использованием целевой переменной [(Наверх)](#0) <a id="3"></a>

In [ ]:
# Построим гистограмму частотного распределения среднего радиуса опухолей
plt.figure(figsize=(10,6))
m = plt.hist(df[df["diagnosis"] == 1].radius_mean, bins=30, fc = (1,0,0,0.5), label = "Злокачественная")
b = plt.hist(df[df["diagnosis"] == 0].radius_mean, bins=30, fc = (0,1,0,0.5), label = "Доброкачественная")
plt.legend()
plt.xlabel("Среднее значение радиуса")
plt.ylabel("Частота")
plt.title("Гистограмма частотного распределения среднего радиуса опухолей")
plt.show()

# Найдем индекс максимальной частоты среднего радиуса злокачественной опухоли
freq_malignant_radius_index = list(m[0]).index(m[0].max())

# Выведем значение высокочастотного среднего радиуса злокачественной опухоли
print('Значение высокочастотного среднего радиуса злокачественной опухоли: {}'.format(m[1][freq_malignant_radius_index]))

## Heatmap для матрицы корреляций [(Наверх)](#0) <a id="4"></a>

In [ ]:
# Рассчитаем корреляции между признаками
corr = df.corr()

# Выведем тепловую карту для корреляций
plt.figure(figsize=(20,12));
sns.heatmap(corr, annot=True, fmt='.2f');

In [ ]:
# Найдем сильноскоррелированные признаки (> 0.75)
CorField = []
for i in corr:
    for j in corr.index[corr[i] > 0.75]:
        if i != j and j not in CorField and i not in CorField:
            CorField.append(j)
            print ("{0} - {1} == {2}".format(i, j, corr[i][corr.index == j].values[0]))

## Попарные scatterplot-ы для сильноскоррелированных признаков [(Наверх)](#0) <a id="5"></a>

In [ ]:
# Построим корреляцию для сильноскоррелированных признаков
corr = df[['diagnosis', 'radius_mean', 'concavepoints_mean', 'perimeter_worst', 'area_mean', 'compactness_mean', 'concavity_mean', 'texture_mean']].corr()  
corr

In [ ]:
# Построим графики парных корреляций
sns.pairplot(df[['diagnosis', 'concavepoints_mean', 'perimeter_worst', 'area_mean', 'compactness_mean', 'concavity_mean', 
                 'radius_mean']], hue="diagnosis");

In [ ]:
# Построим корреляцию для ярко-выраженных признаков
df[['perimeter_worst', 'area_mean', 'radius_mean']].corr()

In [ ]:
# Построим дополнительную корреляцию для ярко-выраженных признаков
df[['compactness_mean', 'concavity_mean']].corr()

In [ ]:
# 0 - доброкачественная, 1 - злокачественная
sns.scatterplot(x='perimeter_worst', y='area_mean', hue='diagnosis', data=df);

In [ ]:
# 0 - доброкачественная, 1 - злокачественная
sns.scatterplot(x='radius_mean', y='area_mean', hue='diagnosis', data=df);

In [ ]:
# 0 - доброкачественная, 1 - злокачественная
sns.scatterplot(x='compactness_mean', y='concavity_mean', hue='diagnosis', data=df);

In [ ]:
# Построим регрессионо-корреляционную прямую для предыдущей зависимости
sns.regplot(x='compactness_mean', y='concavity_mean', data=df)
plt.ylim(0,);

## Отделение злокачественных новообразований от доброкачественных [(Наверх)](#0) <a id="6"></a>

Построим "ящики с усами" для признаков, по которым было бы удобно определять тип опухоли

In [ ]:
sns.boxplot(x='diagnosis', y='perimeter_worst', hue='diagnosis', 
            data=df[['diagnosis', 'perimeter_worst', 'area_mean', 'radius_mean']]);

In [ ]:
sns.boxplot(x='diagnosis', y='area_mean', hue='diagnosis', 
            data=df[['diagnosis', 'perimeter_worst', 'area_mean', 'radius_mean']]);

In [ ]:
sns.boxplot(x='diagnosis', y='radius_mean', hue='diagnosis', 
            data=df[['diagnosis', 'perimeter_worst', 'area_mean', 'radius_mean']]);